# Retrieval API - Step 2 - Generate LLM (Gemini) Answer

## Code Initialization
Dependencies and environment initialization.

Ensure there's a `.env` file with your credentials in the same directory as this script. Use the `.env.example` file as template.

In [1]:
import os
from IPython.display import Markdown
import utils as u
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [2]:
GCP_PROJECT = os.getenv('GCP_PROJECT')
GCP_LOCATION = os.getenv('GCP_LOCATION')
GCP_MODEL = os.getenv('GCP_MODEL')

## Gemini Answer Generation

This is a downstream step that is only illustrative on how the response generation stage can be implemented. In the below example the retrieved articles are used to respond the prompt using an LLM hosted in **Google Cloud Vertex AI**. The passed request is built from the original prompt enhanced with the retrieved articles as gounding context.

The tested LLM is **Google Gemini 2.5 Flash Preview 05-20**. Response generations took a few seconds for a context between 5K to 8K LLM tokens.

### Prerequisite

To successfully complete the following steps, it's important that there's an active GCloud authentication in the environment where this notbeook is executed from. This can be done by running the following command:

```bash
$ gcloud auth application-default login
```

For more information see [Set up ADC for a local development environment](https://cloud.google.com/docs/authentication/set-up-adc-local-dev-environment).

### Read Chunks from [Step 1](1_get_chunks.ipynb)

In [3]:
frapi_query = u.load_dict_json('query')
chunks_list = u.load_dict_json('chunks')
frapi_prompt = frapi_query['data']['attributes']['query']['value']
display(Markdown(f"**Original Prompt:** {frapi_prompt}"))
display(Markdown(f"**Number of Chunks:** {len(chunks_list)}"))

**Original Prompt:** Summarise tariff decisions by the US administration during April and May 2025 and their impact for Tesla

**Number of Chunks:** 10

### Gemini Structured Prompt

In [4]:
instructions_text ="""
    You are an experienced business analyst that respond in a professional manner.
    Answer the query using only the information provided in the list of articles.
    If you use information from an article, cite it using squared brackets containing the index number.
    At the end of the answer, show a list of the cited articles ordered by index and under the title Cited Articles.
    Each cited article must be displayed in the following Markdown format (don't forget the opening square bracket before the headline):
    - [index] [headline - source_name - publication_date](url)
    Use Markdown for the output.
"""
article_list = []

for chunk in chunks_list:
    article = {
        'index': len(article_list) + 1,
        'url': f"https://dj.factiva.com/article?id=drn:archive.newsarticle.{str(chunk['meta']['original_doc_id']).strip()}",
        'source_name': str(chunk['meta']['source']['name']).strip(),
        'headline': str(chunk['attributes']['headline']['main']['text']).strip(),
        'publication_date': chunk['attributes']['publication_date'],
        'content': f"{str(chunk['attributes']['snippet']['content'][0]['text']).strip()} {str(chunk['attributes']['content'][0]['text']).strip()}"
    }
    article_list.append(article)

llm_prompt = {
    'query': frapi_prompt,
    'articles': article_list,
    "instructions": instructions_text.strip()
}

In [5]:
# u.print_full_llm_prompt(llm_prompt)
u.print_partial_llm_prompt(llm_prompt)

{
    "query": "Summarise tariff decisions by the US administration during April and May 2025 and their impact for Tesla",
    "articles": [
        {
            "index": 1,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250508el58004jd",
            "source_name": "Dow Jones Institutional News",
            "headline": "Tesla Stock Got a Lift From Trump's Trade Deal -- Barrons.com",
            "publication_date": "2025-05-08",
            "content": "Al Root Tesla Stock Got a Lift From Trump's Trade Deal -- Barrons.com Al Root Tesla stock rose Thursday as President Donald Trump's trade deal with th..."
        },
        {
            "index": 2,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250506el56000ps",
            "source_name": "Dow Jones Institutional News",
            "headline": "Tesla Stock Falls Amid Tariffs Turmoil. What Ford Earnings Mean for Auto Makers. -- Barrons.com",
            "

### Gemini Request/Response

In [6]:
response = u.gemini_generate(llm_prompt, GCP_PROJECT, GCP_LOCATION, GCP_MODEL)
display(Markdown(f"## **Gemini Model:** {GCP_MODEL}"))
display(Markdown(f"## **Prompt:** {frapi_prompt}"))
display(Markdown(f"## **Gemini Response:**"))
display(Markdown(response))

## **Gemini Model:** gemini-2.5-flash-preview-05-20

## **Prompt:** Summarise tariff decisions by the US administration during April and May 2025 and their impact for Tesla

## **Gemini Response:**

During April and May 2025, the US administration made several tariff-related decisions impacting the automotive industry, including Tesla.

**Tariff Decisions:**
*   In April 2025, Tesla reported that "shifting trade policies, exacerbated by the administration's tariff regime," were stressing supply chains and increasing costs for the automaker. The company was also "contending with 25% tariffs imposed on auto imports" [8, 9, 10].
*   By early May 2025, President Donald Trump "altered his auto tariff policies, giving manufacturers some relief on imported parts" [7].
*   On May 8, 2025, details emerged of a trade deal between President Trump and the U.K., which included "smaller tariffs on imports of vehicles from Britain" [1].
*   Over the weekend preceding May 13, 2025, U.S. and Chinese officials announced a "90-day pause on steep import tariffs" following talks in Switzerland. These tariffs were previously "north of 100%" [3].

**Impact for Tesla:**
*   **Costs and Margins:** Tesla, while assembling all cars sold domestically in the U.S. and having lower direct tariff exposure than many automakers, still faces tariffs on imported parts [2, 7]. Elon Musk noted that "tariffs are still tough on a company when margins are still low" [2, 7]. Tesla's first-quarter operating profit margin was 2%, its weakest since the second quarter of 2019 [2, 7]. The company imports some battery cells from China and over 20% of its components from Mexico, which adds to costs due to tariffs [8, 10].
*   **Sales and Demand:** The ongoing "trade fight" and "changing political sentiment" were cited as factors that could weigh on demand for Tesla's vehicles [8, 9, 10]. President Trump's trade war has led Chinese car buyers to favor local models, impacting Tesla's sales in China [1, 5]. Tesla's sales in China were down about 15% year-over-year over the five weeks ending May 4, and down about 2% in the first quarter of 2025 [1, 5]. Chinese EV sales dropped almost 9% year-over-year in April, and Tesla's Chinese sales were down 56% year-over-year in May (as of May 13) [3]. Weak second-quarter China sales were also linked to U.S.-China trade tensions [6].
*   **Stock Performance:** The announcement of smaller tariffs in the U.S.-U.K. trade deal was seen as a "boon for all auto makers," contributing to a 3.1% rise in Tesla stock on May 8 [1]. Furthermore, the 90-day pause on U.S.-China import tariffs led to a significant boost for Tesla's stock, which rose 6.7% on May 12, as investors welcomed the easing of trade tensions [3].

Cited Articles:
*   [1] [Tesla Stock Got a Lift From Trump's Trade Deal -- Barrons.com - Dow Jones Institutional News - 2025-05-08](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250508el58004jd)
*   [2] [Tesla Stock Falls Amid Tariffs Turmoil. What Ford Earnings Mean for Auto Makers. -- Barrons.com - Dow Jones Institutional News - 2025-05-06](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250506el56000ps)
*   [3] [Tesla Stock Drops on Fall in Chinese Sales, Trump's Plan to Kill EV Credit -- Barrons.com - Dow Jones Institutional News - 2025-05-13](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250513el5d002pj)
*   [5] [Tesla Stock Is Rising. There's Little Evidence of a Post-DOGE Sales Recovery. -- Barrons.com - Dow Jones Institutional News - 2025-05-08](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250508el58002y4)
*   [6] [Tesla Demand Hasn't Rebounded Yet. Watch China This Week. -- Barrons.com - Dow Jones Newswires - 2025-05-18](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250518el5i00017)
*   [7] [Tesla Stock Falls as Sales in Germany Tumble. It's About More Than Musk. -- Barrons.com - Dow Jones Institutional News - 2025-05-06](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250506el56001q4)
*   [8] [Tesla Profit Sinks, Hurt by Backlash Over Elon Musk's Political Role -- Update - Dow Jones Institutional News - 2025-04-22](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250422el4m003rc)
*   [9] [Tesla Profit Sinks, Hurt by Backlash Over Elon Musk's Political Role; CEO says he will spend significantly less time on DOGE starting in May; electric-vehicle maker's net income falls 71% - The Wall Street Journal Online - 2025-04-22](https://dj.factiva.com/article?id=drn:archive.newsarticle.WSJO000020250422el4m0053d)
*   [10] [Tesla Profit Sinks, Hurt by Backlash Over Elon Musk's Political Role -- WSJ - Dow Jones Institutional News - 2025-04-22](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250422el4m003sa)